In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow==1.15
!pip install keras==2.2.5
!pip install git+https://www.github.com/keras-team/keras-contrib.git

     |████████████████████████████████| 412.3MB 35kB/s 
     |████████████████████████████████| 512kB 50.5MB/s 
     |████████████████████████████████| 3.8MB 42.1MB/s 
     |████████████████████████████████| 51kB 5.6MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=ef2c2040245e147b9c11d6c83c83f20bb3c6bedc5d1b756faf9e09e772755b05
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninsta

In [ ]:
import numpy as np

In [ ]:
from keras.models import Sequential, Model, Input, load_model
from keras.layers import LSTM, Dense, TimeDistributed, Activation, Bidirectional, Masking, Embedding, Dropout, Flatten, concatenate, Conv1D, MaxPool1D
from keras import backend as K
from keras_contrib.layers import CRF
from keras.utils import plot_model
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras_contrib.losses import  crf_loss
from keras_contrib.metrics import crf_accuracy

Using TensorFlow backend.


In [ ]:
def building_ner(num_hidden_node, dropout, time_step, vector_length, output_lenght):
    model = Sequential()
    model.add(Masking(mask_value=0., input_shape=(25, time_step, vector_length)))

    model.add(TimeDistributed (LSTM(units=num_hidden_node, return_sequences=False, dropout=dropout, recurrent_dropout=dropout)) )

    model.add( LSTM(units=num_hidden_node, return_sequences=True, dropout=dropout, recurrent_dropout=dropout) )

    model.add(TimeDistributed(Dense(output_lenght)))
    
    model.add(Activation('softmax'))

    plot_model(model, "demo.png", show_shapes=True)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = building_ner(num_hidden_node = 32, dropout = 0.2, time_step = 25, vector_length = 137, output_lenght = 561)
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_2 (Masking)          (None, 25, 25, 137)       0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 25, 32)            21760     
_________________________________________________________________
lstm_4 (LSTM)                (None, 25, 32)            8320      
_________________________________________________________________
time_distributed_4 (TimeDist (None, 25, 561)           18513     
_________________________________________________________________
activation_2 (Activation)    (None, 25, 561)           0         
Total params: 48,593
Trainable params: 48,593
Non-trainable params: 0
_________________________________________________________________


In [ ]:
tag = np.loadtxt('/content/drive/MyDrive/data/tag_encode_input/random_spelling_error_tag_embedd.txt')
print(tag.shape)
char_encode = np.loadtxt('/content/drive/MyDrive/data/char_encode_input/random_spelling_error.txt')
print(char_encode.shape)

(24750, 561)
(24750, 25)


In [ ]:
tag = tag.reshape(int(tag.shape[0]/25), 25, tag.shape[1])
print(tag.shape)
#new
char_encode = char_encode.reshape(int(char_encode.shape[0]/25), 25, char_encode.shape[1])
print(char_encode.shape)

(990, 25, 561)
(990, 25, 25)


In [ ]:
# chuyển từ char encode sang char embedding
LEN_OF_VOCAB = 137
shape = char_encode.shape
char_embedd = np.zeros([shape[0],shape[1],shape[2],LEN_OF_VOCAB])
for i in range(shape[0]):
  for j in range(shape[1]):
    for k in range(shape[2]):
      char_int = char_encode[i,j,k]
      char_int = char_int.astype(np.int64)
      onehot = np.zeros(LEN_OF_VOCAB)
      onehot[char_int] = 1
      char_embedd[i,j,k,:] = onehot

print(char_embedd.shape)

(990, 25, 25, 137)


In [ ]:
import keras

cp_callback = keras.callbacks.ModelCheckpoint(filepath='/content/drive/MyDrive/data/input_output/version_1/random_spelling_error_model', verbose=1, save_best_only=True)
es_callback = keras.callbacks.EarlyStopping(monitor='acc', patience=5)

X_train, X_test, y_train, y_test = train_test_split(char_embedd, tag, test_size=0.1)

# Train the model with the new callback
model.fit(X_train, 
          y_train,
          epochs=600,
          batch_size = 64,
          validation_data=(X_test, y_test),
)
# Pass callback to training

Train on 891 samples, validate on 99 samples
Epoch 1/600
891/891 [==============================] - 9s 10ms/step - loss: 6.1940 - acc: 0.5007 - val_loss: 5.9303 - val_acc: 0.5774
Epoch 2/600
891/891 [==============================] - 7s 8ms/step - loss: 5.5192 - acc: 0.5693 - val_loss: 4.9025 - val_acc: 0.5774
Epoch 3/600
891/891 [==============================] - 6s 7ms/step - loss: 4.4181 - acc: 0.5693 - val_loss: 3.7189 - val_acc: 0.5774
Epoch 4/600
891/891 [==============================] - 6s 7ms/step - loss: 3.3230 - acc: 0.5693 - val_loss: 2.8789 - val_acc: 0.5774
Epoch 5/600
891/891 [==============================] - 6s 7ms/step - loss: 2.7370 - acc: 0.5693 - val_loss: 2.5416 - val_acc: 0.5774
Epoch 6/600
891/891 [==============================] - 6s 7ms/step - loss: 2.5110 - acc: 0.5693 - val_loss: 2.4103 - val_acc: 0.5774
Epoch 7/600
891/891 [==============================] - 6s 7ms/step - loss: 2.4175 - acc: 0.5693 - val_loss: 2.3394 - val_acc: 0.5774
Epoch 8/600
891/891 [==

In [ ]:
model.save("/content/drive/MyDrive/data/input_output/version_1/random_spelling_error_model_1")